# Playground - Possible damage labels

Predict whether "possible damage" labels can be classified as damaged or not

## imports

In [ ]:
import os
os.chdir("..")

In [ ]:
os.getcwd()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import cKDTree
from shapely.geometry import Point
import geopandas as gpd
import json
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score

from labelling_project_config import labelling_options, light_features, medium_features, heavy_features

## Process data, use merge_results.py script, 

**refactor script in master if experiment is succesful?**

In [ ]:
images = "images/images.geojson"

In [ ]:
submissions="submissions"

In [ ]:
damage_features = labelling_options.keys()

In [ ]:
df_image = gpd.read_file(images)
df_image.index = df_image['key']
df_image = df_image.drop(columns=['key'])

In [ ]:
df_image = gpd.read_file(images)
df_image.index = df_image['key']
df_image = df_image.drop(columns=['key'])

damage_features = labelling_options.keys()

os.makedirs(submissions+'-processed', exist_ok=True)
sub_proc_dir = submissions+'-processed'

num_batches = len(list(set([re.findall(r".*?_(\d+)_.*", x)[0] for x in os.listdir(submissions)])))
print('found', num_batches, 'batches')

# loop over batches
for batch in range(num_batches):

    # initialize geodatframe for results
    df_image_batch = df_image.copy()

    # get files and annotator names of this batch
    files = [x for x in os.listdir(submissions) if str(batch) == re.findall(r".*?_(\d+)_.*", x)[0]]
    names = list(set([x.split('.')[0].split('_')[-1] for x in files]))
    num_annotators = len(names)
    map_names_ids = {name: i for i, name in enumerate(names)}
    print('processing batch', batch, 'unique annotators', names)
    batch_keys = []

    # initialize empty columns in results geodatframe
    for i in range(num_annotators):
        for damage_feature in damage_features:
            df_image_batch[damage_feature + '_' + str(i)] = False

    # merge results from different files
    for file in files:

        name = file.split('.')[0].split('_')[-1]
        annotator_id = map_names_ids[name]

        with open(submissions+'/'+file) as json_file:
            data = json.load(json_file)

        for image in data['_via_img_metadata']:
            labels = data['_via_img_metadata'][image]['file_attributes']['damage_labels']
            for damage_feature in damage_features:
                if damage_feature in labels.keys():
                    df_image_batch.at[image, damage_feature+'_'+str(annotator_id)] = True
        batch_keys = list(data['_via_img_metadata'])

    # keep only images in batch
    df_image_batch = df_image_batch[df_image_batch.index.isin(batch_keys)]
    
    # merge results from the diferrent annotators
    for damage_feature in damage_features:
        df_image_batch[damage_feature+'_num_true'] = np.nan
        df_image_batch[damage_feature+'_num_annotator'] = np.nan

    for image in df_image_batch.index:
        df = df_image_batch.loc[image].copy()
        for damage_feature in damage_features:
            labels = [damage_feature+'_'+str(i) for i in range(num_annotators)]
            values = df[labels].values
            #### diff with merge script!!
            df_image_batch.at[image, damage_feature + "_num_true"] = int(sum(values))
            df_image_batch.at[image, damage_feature + "_num_annotator"] = int(len(values))
                
    
    # save processed batch results
    print('finished processing batch', batch, 'with', len(df_image_batch), 'entries')
    if len(df_image_batch) > 0:
        df_image_batch.to_file(sub_proc_dir+'/results_batch_'+str(batch)+'.geojson', driver='GeoJSON')

In [ ]:
# merge results from all batches
gdf_results = gpd.GeoDataFrame()
for file in os.listdir(sub_proc_dir):
    gdf_batch = gpd.read_file(sub_proc_dir+'/'+file)
    gdf_results = gdf_results.append(gdf_batch, ignore_index=True)

## Experiment: can we predict the consensus labels

### Prepare the data

In [ ]:
# try out with one damage class as target class
model_damage_class = "severe_structural_damage"

In [ ]:
cols = ["key", "geometry"]
for damage_class in damage_features:
    cols.append(f"{damage_class}_num_true")
    cols.append(f"{damage_class}_num_annotator")

gdf_sub = gdf_results[cols]

In [ ]:
for damage_class in damage_features:
    gdf_sub.loc[:, f"{damage_class}_num_true"] = gdf_sub.loc[:, f"{damage_class}_num_true"].astype('int')
    gdf_sub.loc[:, f"{damage_class}_num_annotator"] = gdf_sub.loc[:, f"{damage_class}_num_annotator"].astype('int')

In [ ]:
# create labels: all true = 1, all false = 0, else unknown, coded as 2

for damage_class in damage_features:
    gdf_sub.loc[:, f"{damage_class}_label"] = 2

    gdf_sub.loc[gdf_sub[f'{damage_class}_num_true'] == gdf_sub[f'{damage_class}_num_annotator'],\
                f"{damage_class}_label"] = 1
    gdf_sub.loc[gdf_sub[f'{damage_class}_num_true'] == 0, \
                f"{damage_class}_label"] = 0


In [ ]:
# plot 
fig, ax = plt.subplots(figsize=(15,20))
gdf_sub.loc[gdf_sub[f'{damage_class}_label'] == 0].geometry.plot(color='blue',\
                                                                 ax=ax, alpha=0.3,\
                                                                 label="no damage")
gdf_sub.loc[gdf_sub[f'{damage_class}_label'] == 2].geometry.plot(color='orange',\
                                                                 ax=ax, alpha=0.5,\
                                                                 label="possible damage")
gdf_sub.loc[gdf_sub[f'{damage_class}_label'] == 1].geometry.plot(color='red',\
                                                                 ax=ax,\
                                                                 label="damage")
ax.legend()

In [ ]:
# create features from gps points
gdf_sub["x_coord"] = gdf_sub.geometry.x
gdf_sub["y_coord"] = gdf_sub.geometry.y

In [ ]:
# known labels to train and test model
# apply model to unknown labels 

model_cols = ["x_coord", "y_coord"]
for damage_class in damage_features:
    model_cols.append(f"{damage_class}_label")

train_test_gdf = gdf_sub.loc[gdf_sub[f"{model_damage_class}_label"] != 2][model_cols]

predict_gdf = gdf_sub.loc[gdf_sub[f"{model_damage_class}_label"]== 2][model_cols]

In [ ]:
# cast label to ints
for damage_class in damage_features:
    train_test_gdf.loc[:, f"{damage_class}_label"] = train_test_gdf.loc[:, f"{damage_class}_label"].astype('int')
    predict_gdf.loc[:, f"{damage_class}_label"] = predict_gdf.loc[:, f"{damage_class}_label"].astype('int')
    
#shuffle data
train_test_gdf = train_test_gdf.sample(frac=1)

In [ ]:
# split data into train and test
frac_predict = 0.2
predict_size = int(train_test_gdf.shape[0]*frac_predict)

train_df = train_test_gdf[:-predict_size]
test_df = train_test_gdf[-predict_size:]

# split into features and target
target_list = [f"{model_damage_class}_label"]
train_X = train_df.drop(columns=target_list)
train_y = train_df[f"{model_damage_class}_label"]
test_X = test_df.drop(columns=target_list)
test_y = test_df[f"{model_damage_class}_label"]

# do the same for all unknown labels for predicting
predict_X = predict_gdf.drop(columns=target_list)

## Random Forest

In [ ]:
clf = RandomForestClassifier(n_estimators=10)

In [ ]:
# Cross validation score on train data
scores = cross_val_score(clf, train_X, train_y, cv=5)
scores.mean()

In [ ]:
# fit RF
clf = clf.fit(train_X, train_y)

In [ ]:
# Score on unseen data
# check it's in the same order as average cross validation score
clf.score(test_X, test_y)

In [ ]:
# use model to predict whether the "possible" damage labels are yes or no
predict_y = clf.predict(predict_X)

In [ ]:
# transform to geodataframe for plotting
predict_X[f"{model_damage_class}_label"] = predict_y

gdf = gpd.GeoDataFrame(
        predict_X, 
        geometry=gpd.points_from_xy(predict_X.x_coord, predict_X.y_coord))

In [ ]:
### Plots

# plot 
fig, ax = plt.subplots(figsize=(15,20))
gdf.loc[gdf[f'{model_damage_class}_label'] == 0].geometry.plot(color='blue',\
                                                               ax=ax,\
                                                               alpha=1,\
                                                               marker="8",\
                                                              label="predicted - no damage")

gdf.loc[gdf[f'{model_damage_class}_label'] == 1].geometry.plot(color='red',\
                                                               ax=ax,\
                                                               marker="8",\
                                                              label="predicted - damage")

gdf_sub.loc[gdf_sub[f'{model_damage_class}_label'] == 1].geometry.plot(color='red',\
                                                                       ax=ax,\
                                                                       alpha=0.2,\
                                                                       label="groundtruth - damage",\
                                                                      marker="+")

gdf_sub.loc[gdf_sub[f'{model_damage_class}_label'] == 0].geometry.plot(color='blue',\
                                                                       ax=ax,\
                                                                       alpha=0.2,\
                                                                      label="groundtruth - damage",\
                                                                      marker="+")

ax.legend()


### Evaluate what is the distance between "possible damage" points to its nearest neighbour with the same label as predicted label"

and compare with distance between nearest neighbours with the same label

In [ ]:
def ckdnearest(gdA, gdB, k=1):
    """
    Calculates for each point in gdB the distance to its kth nearest neighbour in gdA
    
    gdA : gpd.DataFrame
    gdB: gpd.DataFrame
    k: kth nearest neighbour, default=1, if gdA=gdB, then exclude distance to itself by choosing k=[2]
    
    return gdB with distance to kth nearest neighbour
    """
    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=k)
    return gdB.reset_index(drop=True).join(pd.Series(dist.flatten(), name='dist'))


In [ ]:
def gpd1(damage):
    return gdf_sub.loc[gdf_sub[f'{model_damage_class}_label'] == damage]

def gpd2(damage):
    return gdf.loc[gdf[f'{model_damage_class}_label'] == damage]

# boxplots of distance
fig, axs = plt.subplots(1,2)
axs[0].boxplot([ckdnearest(gpd1(0), gpd2(0)).dist,\
                ckdnearest(gpd1(0), gpd1(0), k=[2]).dist],\
               labels=["predicted", "groundtruth"])

axs[1].boxplot([ckdnearest(gpd1(1), gpd2(1)).dist.dropna(),\
                ckdnearest(gpd1(1), gpd1(1), k=[2]).dist],\
               labels=["predicted", "groundtruth"])
axs[0].set_title("no damage")
axs[1].set_title("damage")

## KMeans

In [ ]:
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(train_test_gdf)

In [ ]:
y_clusters = kmeans.labels_

In [ ]:
# plot 
fig, ax = plt.subplots(figsize=(20,5))

for cluster in range(n_clusters):
    plt.scatter(
    x=train_test_gdf.loc[(y_clusters == cluster), "x_coord"],
    y=train_test_gdf.loc[(y_clusters == cluster), "y_coord"])
